# Advent of Code 2024 Day 12 

### Setup

In [103]:
from aocd import get_data, submit

day = 12
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
import numpy as np 

def parse_data(raw_data:str):
    return np.reshape(np.array([[c] for row in raw_data.split() for c in row]), (len(raw_data.split()), len(raw_data.split()[0])))

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data.shape

### Part One!

In [107]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [109]:
def is_out_of_bounds(pos, max_x, max_y):
    return pos[0] < 0 or pos[0] >= max_x or pos[1] < 0 or pos[1] >= max_y

In [110]:
def get_neighbor_indices(position: tuple[int, int], max_x: int, max_y: int, ignore_out_of_bounds: bool = False):
    neighbors = []

    if position[0] + 1 < max_x or ignore_out_of_bounds:
        neighbors.append((position[0] + 1, position[1]))
    
    if position[0] - 1 >= 0 or ignore_out_of_bounds:
        neighbors.append((position[0] - 1, position[1]))
    
    if position[1] + 1 < max_y or ignore_out_of_bounds:
        neighbors.append((position[0], position[1] + 1))

    if position[1] - 1 >= 0 or ignore_out_of_bounds:
        neighbors.append((position[0], position[1] - 1))
    
    return neighbors


In [111]:
import heapq

def get_areas(data: np.ndarray):
    areas = {}
    for groupkey in np.unique(data):
        areas.setdefault(groupkey, [])
        indices = np.argwhere(data == groupkey)
        
        for index in (tuple(x) for x in indices):
            matching_neighbors = [x for x in get_neighbor_indices(index, data.shape[0], data.shape[1]) if data[x] == groupkey]
            assignments = []
            for neighbor in matching_neighbors:
                for group_idx, group in enumerate(areas[groupkey]):
                    if group_idx in assignments:
                        continue
                    
                    if neighbor in group:
                        assignments.append(group_idx)
                        continue

            if not assignments:
                areas[groupkey].append({index})

            elif len(assignments) == 1:
                areas[groupkey][assignments[0]].add(index)
            
            elif len(assignments) > 1:
                combined = set([index])
                for idx in assignments:
                    combined.update(areas[groupkey][idx])
                
                areas[groupkey] = [x for idx, x in enumerate(areas[groupkey]) if idx not in assignments]
                areas[groupkey].append(combined)
                    
    return areas

In [112]:
def get_perimeters(data:np.array, areas: dict):
    perimeters = {}

    for groupkey, group in areas.items():
        perimeters.setdefault(groupkey, [])
        for area in group:
            area_perimeter = 0
            for position in area:
                for neighbor in get_neighbor_indices(position, *data.shape, ignore_out_of_bounds=True):
                    if neighbor not in area or is_out_of_bounds(neighbor, *data.shape):
                        area_perimeter += 1
                
            perimeters[groupkey].append(area_perimeter)
                    
    return perimeters
    

In [113]:
def calculate_area_perimeter_product(areas: dict, perimeters: dict):
    sum = 0
    for groupkey, group in areas.items():
        for idx, area in enumerate(group):
            sum += len(area) * perimeters[groupkey][idx]

    return sum

In [ ]:
areas = get_areas(data)
perimeters = get_perimeters(data, areas)

part_a_answer = calculate_area_perimeter_product(areas, perimeters)
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [123]:
use_sample_data = False
part='b'

In [ ]:
data = sample_data if use_sample_data else test_data
data

In [125]:
import numpy as np 

def get_area_sides(data:np.ndarray, points: set):
    count = 0

    # count top side (loop through all rows)
    for row_idx, row in enumerate(data):
        is_first_row = row_idx == 0
        last = None
        for col_idx, col in enumerate(row):
            if (row_idx, col_idx) in points:
                is_area_above = (row_idx - 1, col_idx) in points
                is_continuation = last == (row_idx, col_idx - 1) and (is_first_row or (row_idx - 1, col_idx - 1) not in points)

                if not is_area_above and not is_continuation:
                    count += 1

                last = (row_idx, col_idx)
    
                
    # count bottom sides (loop through all rows)
    for row_idx, row in enumerate(data):
        is_last_row = row_idx == len(data) - 1
        last = None
        for col_idx, col in enumerate(row):
            if (row_idx, col_idx) in points:
                is_area_below = (row_idx + 1, col_idx) in points
                is_continuation = last == (row_idx, col_idx - 1) and (is_last_row or (row_idx + 1, col_idx - 1) not in points)

                if not is_area_below and not is_continuation:
                    count += 1

                last = (row_idx, col_idx)
    
    # count left side (loop through all columns)
    for col_idx in range(len(data[0])):
        is_first_col = col_idx == 0
        last = None
        for row_idx in range(len(data)):
            if (row_idx, col_idx) in points:
                is_area_left = (row_idx, col_idx - 1) in points
                is_continuation = last == (row_idx - 1, col_idx) and (is_first_col or (row_idx - 1, col_idx - 1) not in points)

                if not is_area_left and not is_continuation:
                    count += 1

                last = (row_idx, col_idx)
    
    # count right side (loop through all columns)
    for col_idx in range(len(data[0])):
        is_last_col = col_idx == len(data[0]) - 1
        last = None
        for row_idx in range(len(data)):
            if (row_idx, col_idx) in points:
                is_area_right = (row_idx, col_idx + 1) in points
                is_continuation = last == (row_idx - 1, col_idx) and (is_last_col or (row_idx - 1, col_idx + 1) not in points)

                if not is_area_right and not is_continuation:
                    count += 1

                last = (row_idx, col_idx)
    
    
    return count
    

In [126]:
def get_sides(data, areas):
    sides = {}
    for groupkey, group in areas.items():
        sides.setdefault(groupkey, [])
        for area in group:
            sides[groupkey].append(get_area_sides(data, area))
    
    return sides

In [127]:
def calculate_area_side_product(areas: dict, sides: dict):
    sum = 0
    for groupkey, group in areas.items():
        for idx, area in enumerate(group):
            sum += len(area) * sides[groupkey][idx]

    return sum

In [ ]:
areas = get_areas(data)
sides = get_sides(data, areas)

part_b_answer = calculate_area_side_product(areas, sides)
part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)